In [2]:
def get_genome_file(genome_location,species):
    list_of_files_in_genome_folder = os.listdir(f"{genome_location}/{species}")
    for file in list_of_files_in_genome_folder:
        if file.endswith("_genomic.fna"):
            genome_file = file
    return(genome_file)  

def get_gene_sequence(genome_location, 
                      species, 
                      genome_file, 
                      scaffold,
                      gene_start,
                      gene_end,
                      complement,
                      output_location):

    
    print("Getting Gene")
    list_of_folders = os.listdir(output_location)
    # print(list_of_files_inside_annotated_species_folder)
    # print(
    if f"0.Temp" not in list_of_folders:
        os.mkdir(f"{output_location}/0.Temp")

#     subprocess.run(f'samtools faidx "{genome_location}/{species}/{genome_file}"', shell = True, stderr = subprocess.DEVNULL)
#     print(f'samtools faidx "{genome_location}/{species}/{genome_file}" {scaffold}:{gene_start}-{gene_end}')
    subprocess.run(f'samtools faidx "{genome_location}/{species}/{genome_file}" {scaffold}:{gene_start}-{gene_end} > "{output_location}/0.Temp/temp_genome.fa"', shell = True, stderr = subprocess.DEVNULL)
    
    genome = SeqIO.parse(f"{output_location}/0.Temp/temp_genome.fa", "fasta")
    for entries in genome:
        gene_sequence = entries.seq
        if complement == "1":
            gene_sequence = gene_sequence.reverse_complement()
        break
    # print(gene_sequence)
    return (gene_sequence)
def make_output_folder(output_location):
    list_of_folders = os.listdir(output_location)
    if "2.Final_output" in list_of_folders:
        subprocess.run(f'rm -r "{output_location}/2.Final_output"', shell = True, stderr = subprocess.DEVNULL)
    os.mkdir(f"{output_location}/2.Final_output")

def write_exon_output(output_location,species,exon,exon_output):
    
    
    list_of_folders_in_final_output = os.listdir(f"{output_location}/2.Final_output")
    if species not in list_of_folders_in_final_output:
        os.mkdir(f"{output_location}/2.Final_output/{species}")
    
    with open(f"{output_location}/2.Final_output/{species}/{exon}.fa", 'w') as out_file:
        out_file.write(exon_output)

In [6]:
import os
from Bio import SeqIO
import subprocess

family_group = "6.Heliconiinae_Danainae_Nymphalinae"


output_location = f"/mnt/h/My Drive/Circadian Rhythm Genes Project/9.CLOCK Exon Analysis/{family_group}"
make_output_folder(output_location)
genome_location = "/mnt/f/Genomes_2023-12-26"
list_of_species = os.listdir(f"{output_location}/1.Blast_result")
if 'desktop.ini' in list_of_species:
    list_of_species.remove("desktop.ini")
# print(list_of_species)
# list_of_species = ["Heliconius_charithonia"]
# list_of_species = ["Maniola_hyperantus"]
error_exons = ''
for species in list_of_species:
    print(species)
    genome_file = get_genome_file(genome_location,species)
    print(genome_file)
    previous_coordinate_start = ''
    previous_scaffold = ''
    with open(f"{output_location}/1.Blast_result/{species}/final_coordinates.csv", 'r') as coordinate_final_file:
        coordinate_file_list = coordinate_final_file.readlines()
    last_exon = len(coordinate_file_list)-1
    
    for i in range(1,len(coordinate_file_list)):
        
        coordinate_line_split = coordinate_file_list[i].split(",")
        
            
#         print(coodinate_line_split[5])
        if coordinate_line_split[5] == "N":
            if previous_coordinate_start == '':
                previous_coordinate_start = coordinate_line_split[2]
                previous_scaffold = coordinate_line_split[1]
            complement = coordinate_line_split[4]
            start,stop = coordinate_line_split[2],coordinate_line_split[3]
            print(start,stop)
            query_name = coordinate_line_split[6]
            scaffold = coordinate_line_split[1]
            exon = f"Exon_{query_name.split('_')[-1]}"
            left_oh, right_oh = query_name.split('_')[-6], query_name.split('_')[-4]
            print(left_oh, right_oh)
#             print(complement)
            if complement == "0":
                if start < previous_coordinate_start or scaffold != previous_scaffold:
                    print(f"Error in Coordinates\nPrevious_coordinate = {coordinate_file_list[i-1]}\nCurrent_coordinate = {coordinate_file_list[i]}")
                    prompt = input("Continue (Y/N)?")
                    if prompt[0].lower() == "n":
                        assert False
            elif complement == "1":
                if start > previous_coordinate_start or scaffold != previous_scaffold:
                    print(f"Error in Coordinates\nPrevious_coordinate = {coordinate_file_list[i-1]}\nCurrent_coordinate = {coordinate_file_list[i]}")
                    prompt = input("Continue (Y/N)?")
                    if prompt[0].lower() == "n":
                        assert False
            else:
                print("ERROR")
            
        elif coordinate_line_split[5] == "Y":
            query_name = coordinate_line_split[6]
            exon = f"Exon_{query_name.split('_')[-1]}"
            scaffold = coordinate_line_split[1]
            error_exons += f"{species},{scaffold},000,000,0,Y,Error_{exon},00,00,00\n"
            continue
            
        gene_sequence = get_gene_sequence(genome_location, 
                                          species, 
                                          genome_file, 
                                          scaffold,
                                          start,
                                          stop,
                                          complement,
                                          output_location)
        
       
        current_exon = query_name.split('_')[-1]
        print(f"Current Exon = {current_exon}")
        if current_exon == "1":
            if complement == 1:
                translated = gene_sequence.reverse_complement.translate()
            else:
                translated = gene_sequence.translate()
            print(translated)
            if "*" in translated and translated[0] != "M":
                error_exons += f"{species},{scaffold},000,000,0,Y,Error_{exon},00,00,00\n"
            else:
                exon_output = f">{species}_{exon}_{scaffold}_{start}_{stop}_left_{left_oh}_right_{right_oh}\n{gene_sequence}"
                print(exon_output)
        
                write_exon_output(output_location,species,exon,exon_output)
        elif current_exon == f"{last_exon}":
            if complement == 1:
                translated = gene_sequence[:-int(left_oh)].reverse_complement.translate()
                print(gene_sequence)
            else:
                translated = gene_sequence[int(left_oh):].translate()
                print(gene_sequence)

            if "*" in translated[:-1] and translated[-1] != "*":
                error_exons += f"{species},{scaffold},000,000,0,Y,Error_{exon},00,00,00\n"
                print(translated)
            else:
                
                exon_output = f">{species}_{exon}_{scaffold}_{start}_{stop}_left_{left_oh}_right_{right_oh}\n{gene_sequence}"
                print(exon_output)
                write_exon_output(output_location,species,exon,exon_output)
#         assert False
        else:
            exon_output = f">{species}_{exon}_{scaffold}_{start}_{stop}_left_{left_oh}_right_{right_oh}\n{gene_sequence}"
            print(exon_output)

            write_exon_output(output_location,species,exon,exon_output)
        
#         assert False
        previous_coordinate_start = start
        previous_scaffold = scaffold
    error_exons += "\n\n"
#     print(coordinate_file_list)
#     break
coordinate_file = ''
print(error_exons)
with open(f"{output_location}/error_exons.txt", 'w') as error_out_file:
    error_out_file.write(error_exons)

Hestina_assimilis
GCA_023373885.1_ASM2337388v1_genomic.fna
30880775 30880812
0 2
Getting Gene
Current Exon = 1
MEDDGDDKDDSK
>Hestina_assimilis_Exon_1_CM041759.1_30880775_30880812_left_0_right_2
ATGGAGGATGATGGAGATGACAAAGATGATTCTAAAag
30882230 30882378
1 1
Getting Gene
Current Exon = 2
>Hestina_assimilis_Exon_2_CM041759.1_30882230_30882378_left_1_right_1
GAAAACTCGGAACTTAAGTGAAAAGAAAAGGCGAGATCAATTCAATATGTTGGTTAATGAACTGGGTTCTATGGTTTCCACAAACAACAGAAAAATGGATAAATCTACTGTACTAAAATCtacaatatcatttttaaaaaatcataatg
30882451 30882632
2 0
Getting Gene
Current Exon = 3
>Hestina_assimilis_Exon_3_CM041759.1_30882451_30882632_left_2_right_0
aAGTTACTATTCGCTCAAGAGCTCATGATGTACAGGAAGATTGGAAACCAGCTTTTCTTTCAAATGAAGAATTTACATATTTAGTACTGGAGGCATTGGAAGGATTTGTTATGGTCTTCTCTTCTTCTGGTTGTATACACTATGTCTCGGAAAgcattttatcattattaggGCTCAATCCG
30882712 30882829
0 1
Getting Gene
Current Exon = 4
>Hestina_assimilis_Exon_4_CM041759.1_30882712_30882829_left_0_right_1
AACAATGTagttaataaaagtatatttgaaCTGGCGCATGAAGAAGATCATTCcaacttatataatat

3290327 3290475
1 1
Getting Gene
Current Exon = 2
>Danaus_plexippus_Exon_2_NC_083559.1_3290327_3290475_left_1_right_1
GAGAACTCGCAATCTTAGTGAGAAGAAGAGGAGAGACCAGTTCAACATGCTTGTCAATGAACTCGGTTCTATGGTATCaacaaataatagaaaaatggaTAAATCAACTGTTCTTAAGTCTACTATATCATTCTTAAAGAACCATAATG
3290030 3290211
2 0
Getting Gene
Current Exon = 3
>Danaus_plexippus_Exon_3_NC_083559.1_3290030_3290211_left_2_right_0
AGATAACAGTGCGATCACGAGCTCATGATGTCCAAGAAGATTGGAAGCCGGCTTTCTTATCTAATGAGGAATTCACCTACTTAGTCTTGGAGGCCTTGGAGGGTTTTGTTATGGTATTTTCAGCTAGCGGTTGCATTTACTATGTATCGGAAAGTGTGACATCTTTGCTAGGGCATACTCCG
3289833 3289947
0 1
Getting Gene
Current Exon = 4
>Danaus_plexippus_Exon_4_NC_083559.1_3289833_3289947_left_0_right_1
GGAGACATtatcaataaaagtatatttgatTTGGCATTTGTTGACGATCGTCCAAATCTATACAACATTTTGCAAAATGGTGGTACCCTTGATCCGACCCAAGTTGTGACGACag
3288152 3288247
2 1
Getting Gene
Current Exon = 5
>Danaus_plexippus_Exon_5_NC_083559.1_3288152_3288247_left_2_right_1
ATAATCCTATAAGCTTCCGTTGCCGCCTGCAAAGGGGAACATTAGATTTCAGAGATGAAGTAACCTACGAATTGGTCCA

Current Exon = 7
>Mechanitis_mazaeus_Exon_7_OY365748.1_16697909_16698147_left_1_right_1
GTTGCTTTTCGTGTGCACAGGCAGGCTATACACTCCCCAACTAGTTCGAGATGTGTCTCTAGTTGATACTATTCGTAGTGAATTCACATCCAGACATAGCCTAGAGTGGAAGTTCTTGTTCCTGGATCACCGCGCTCCCCCAATTATCGGCTACTTGCCATTCGAAGTTCTGGGAACTTCTGGATACGATTACTATCATTTTGATGATCTGGAGAAAGTTGTCTCCTGTCATGAAGCTT
16698733 16698880
2 2
Getting Gene
Current Exon = 8
>Mechanitis_mazaeus_Exon_8_OY365748.1_16698733_16698880_left_2_right_2
TGATGCAAAAAGGGGAGTTGACGTCTTGCTTCTACAGATTTTTGACTAAGGGTCAACAATGGATCTGGCTTCAGACACGTTTCTACATAACATACCATCAGTGGAACTCCAAGCCAGAGTTTGTGGTTTGCACTCATCGTGTtgttag
16698975 16699158
1 0
Getting Gene
Current Exon = 9
>Mechanitis_mazaeus_Exon_9_OY365748.1_16698975_16699158_left_1_right_0
CTATGCTGATATAGTCAAAACCACAAAACAGGAGAGCGCAGAGACAGAGACAGCTTTGGAGGCTGAAGAGAGCCGCGCTATGATGAAGGACACCTCCACCGAAGACGCCATGGTGCCCGTGTCACCATCATATATGTCGGAAGCGAGTGATGCTTTTGGAAACTCATACCACTCTTTATCAAAG
16700040 16700228
0 0
Getting Gene
Current Exon = 10
>Mechanitis_mazaeus_Exon_10_OY365748.1_

Current Exon = 10
>Melinaea_marsaeus_rileyi_Exon_10_OU911095.1_15470908_15471096_left_0_right_0
CCGGCGTCAGTGAAATCGGCGGCCACGTCGGCTGGCAGCACCAGTGCCACTGTTGCGACCTTGGGCACCACCACCACAGCAGGGGCGTCCTGGCCGACTCGCTCCTCCTACCTGTTGTACACCACGGGGTCAGACACGGCCTCAGTGTCTGGGGGGTCGCGGTCGTCCCAGCGTAATAGCTCTCACGAG
15471378 15471539
0 0
Getting Gene
Current Exon = 11
>Melinaea_marsaeus_rileyi_Exon_11_OU911095.1_15471378_15471539_left_0_right_0
GTGCAAAGAGTATCAGAGCCGGCGCTGGTGCCGCAGCACGGTATCGGGGCTCAATACCTGGAGCCGGCTCCGTACGTGGGTGCCGTGGGTGTACCCGCAGTGCTGCCGCTACCCCTACCACCGATACCAGTCATCGTGACCCCCGACCAGGCCCAGCTTCAg
15471732 15471854
0 0
Getting Gene
Current Exon = 12
>Melinaea_marsaeus_rileyi_Exon_12_OU911095.1_15471732_15471854_left_0_right_0
GAGCAGTTGCAGCGGACTCACCGCGAGTTACAGCAGATGATCGTACGGCAGCAGGAGGAGCTGAGGCAGGTTAAGGAGCAGCTGCTGTTCGCGAGGCTCGGCATACTCCAGCCGATCATTAAC
15472540 15472671
0 0
Getting Gene
Current Exon = 13
GTGCAGGATCCCTATTCAAACCCGGAACAGCTCCAGAGTCGACCGGCGTCATCCATCATGTACGAGGGGAACAGACAGCTCGGTTACCCACCCCCAAACAACCAACAGAACC

Current Exon = 1
MDDDGDDKDDTK
>Napeogenes_sylphis_Exon_1_OY365784.1_28824603_28824640_left_0_right_2
ATGGACGACGACGGAGATGATAAAGATGACACTAAaag
28825248 28825396
1 1
Getting Gene
Current Exon = 2
>Napeogenes_sylphis_Exon_2_OY365784.1_28825248_28825396_left_1_right_1
GAGGACTCGTAACCTTAGTGAGAAGAAGAGGAGAGATCAGTTCAACATGCTTGTTAATGAACTTGGTTCCATGGTATCAACTAATAATAGAAAGATGGATAAATCTACTGTTCTTAAGTCCACTATATCATTCTTAAAAAACCATAATG
28825704 28825885
2 0
Getting Gene
Current Exon = 3
>Napeogenes_sylphis_Exon_3_OY365784.1_28825704_28825885_left_2_right_0
AAATAACTGTACGATCACGAGCCCATGATGTACAAGAAGATTGGAAGCCAGCCTTTTTGTCAAATGAAGAATTCACATACTTAGTATTGGAGGCTTTAGAAGGATTTGTGATGGTATTTTCAGCAACTGGTTGCATCTACTATGTATCTGAAAGTGTCACATCTTTACTAGGACACACTCCG
28826278 28826383
0 1
Getting Gene
Current Exon = 4
>Napeogenes_sylphis_Exon_4_OY365784.1_28826278_28826383_left_0_right_1
ggtGACATAATCAATAAGAGTATATTTGACTTGGCATTTGAGGATGATCGTCAAAGTTTGTACAGCCTTCTGCAAAACCCTGGTACATCTCTTGACCCAACACAAG
28828274 28828369
2 1
Getting Gene
Current Exon = 5

...

Current Exon = 1
MDDDGDDKDDSK
>Euphydryas_editha_Exon_1_CAKOGL010000026.1_1931287_1931324_left_0_right_2
ATGGACGACGACGGAGATGACAAAGATGACTCGAAAag
1930478 1930626
1 1
Getting Gene
Current Exon = 2
>Euphydryas_editha_Exon_2_CAKOGL010000026.1_1930478_1930626_left_1_right_1
GAGGACACGTAACTTAAGTGAAAAAAAGAGGCGAGACCAGTTCAATATGCTTGTTAATGAACTTGGATCTATGGTGTCGACTAATAATAGAAAGATGGACAAATCTACTGTCCTTAAATcaacaatttcttttttaaagaaTCATAACG
1930208 1930389
2 0
Getting Gene
Current Exon = 3
>Euphydryas_editha_Exon_3_CAKOGL010000026.1_1930208_1930389_left_2_right_0
AAATAACGATTCGTTCGAGAGCTCACGATGTACAAGAAGATTGGAAACCTGTATTTTTGTCAAATGAAgaatttacttatttagtaCTGGAGGCATTAGAAGGATTTGTTTTGGTGTTTTCAGCAACTGGTCAAATCTACTATGTCTCAGAAAGTATAATGTCCTTATTAGGCCATAATCcg
1930011 1930128
0 1
Getting Gene
Current Exon = 4
>Euphydryas_editha_Exon_4_CAKOGL010000026.1_1930011_1930128_left_0_right_1
AATGATATAGTGAACAGAAGTATATTTGATTTGGTATTTGAGGATGATCGTCCTAACTTATATGGCTTACTACAGAATCCTGCTGCAGCATTACACCCTACACAAACAGTAACATCAG
1925562 1925657
2 1
Getting G

Current Exon = 4
>Melitaea_cinxia_Exon_4_NC_059424.1_8955458_8955575_left_0_right_1
AGTAACATAGTAAACAAAAGTATATTTGATTTAGTATTTGAGGATGATCGTCCTAACTTGTACGGCTTACTACAGAATCCTGTTCCCGTAATACATCCTTCTCAAACAGTAACATCag
8963331 8963426
2 1
Getting Gene
Current Exon = 5
>Melitaea_cinxia_Exon_5_NC_059424.1_8963331_8963426_left_2_right_1
agaATGAAATAAGATTTCAGTGCCATTTACAAAGAGGTACATTGGACTTTCGAGATGAGACTACGTATGAACTCGTCCAGTTCAACGGACACTTCC
8967350 8967410
2 2
Getting Gene
Current Exon = 6
>Melitaea_cinxia_Exon_6_NC_059424.1_8967350_8967410_left_2_right_2
GTACATATATGGAACAAAACGAAGGTCCTAACGTTCTGTATCAACAAAATCAGGACTCGAG
8970704 8970942
1 1
Getting Gene
Current Exon = 7
>Melitaea_cinxia_Exon_7_NC_059424.1_8970704_8970942_left_1_right_1
GTTACTGTTCGTCTGTACTGGAAAACTTTGCACACCGCAATTGCTTCGCGACGTGTCTCTCGTGGACTCAAATCGAAGCGAGTTCACATCTCGTCATAGTTTGGAATGGAAATTCCTTTTTCTGGATCATCGTGCACCGCCTATCATTGGCTACTTGCCGTTCGAAGTGCTCGGCACTTCAGGATACGACTACTACCATTTTGACGATTTGGAGAAAGTCGTGACGTGCCATGAAGCTC
8971534 8971681
2 2
Getting Gene
Current Exon =

Current Exon = 9
>Nymphalis_c-album_Exon_9_OY720586.1_11572607_11572790_left_1_right_0
TTATGAAGATATGTATAAAGGTGCGAAGCATGAAAACGTTGATACTGAAACCGCAAGTGAAACGGAACCTAAAATAAGTGACATGAAAGAGGCATCATCAGAAGATGCTGTTCTGGTAATGTCACCATCGTATATGTCAGACCCAAGTGAAGCTTTCGGGTCGTCGTACCAATCTTTGTCACag
11573487 11573657
0 0
Getting Gene
Current Exon = 10
>Nymphalis_c-album_Exon_10_OY720586.1_11573487_11573657_left_0_right_0
CCACCATCGGTGAAATCGACCGCGGCGTCAACTGGCGACACTGTGCCAACTATGGCGACCACTGCCACCTCCTCGTGGCCCACTTGCGCGTCCCAGATGATGTTCTCGGCCGGGTCCGATACGGCCTCAGTGTCGGGCGACTCGCGATCGCCGCGCCTGGTCGCGCAAGAT
11573783 11573944
0 0
Getting Gene
Current Exon = 11
>Nymphalis_c-album_Exon_11_OY720586.1_11573783_11573944_left_0_right_0
CTGCCTAGGTTACCGCAACCGGCGATAGTACCCCAGCACGTTATGGGCGCGCAGTACTTGGAGCCGGGACCGTACATTGGATCCGTCGGCGTCTCGGGGGTGTTGACGCTCCCGATGCACTCTGTGCCGCTGATCGTCACCCCGGATCAAGCGCAGTTGCAG
11574297 11574419
0 0
Getting Gene
Current Exon = 12
>Nymphalis_c-album_Exon_12_OY720586.1_11574297_11574419_left_0_right_0
GAGCAGTTGCAGCGCACACAT

11407818 11408018
0 0
Getting Gene
Current Exon = 13
gTTCAAGATCCATTCACTCATACCGAACAAATGCAACAAAACGAGCTGCTGCCAGCACAAATGCTTTACGACGGAACAGCCGCTAGACCTGTACCCAGCTACCCGCACCACccaccgccgccgccgccgggTGGTCACCACATCAACACGCATCAAAATCAAAACCAAAATCATCATAATCATATGCCGCCGCAGCTGTAA
>Nymphalis_polychloros_Exon_13_HG992241.1_11407818_11408018_left_0_right_0
gTTCAAGATCCATTCACTCATACCGAACAAATGCAACAAAACGAGCTGCTGCCAGCACAAATGCTTTACGACGGAACAGCCGCTAGACCTGTACCCAGCTACCCGCACCACccaccgccgccgccgccgggTGGTCACCACATCAACACGCATCAAAATCAAAACCAAAATCATCATAATCATATGCCGCCGCAGCTGTAA
Nymphalis_urticae
GCA_905147175.2_ilAglUrti1.2_genomic.fna
6509095 6509132
0 2
Getting Gene
Current Exon = 1
MDDDGDDKDDSK
>Nymphalis_urticae_Exon_1_LR989982.1_6509095_6509132_left_0_right_2
ATGGACGACGACGGTGATGATAAAGATGACTCGAAAAG
6508152 6508300
1 1
Getting Gene
Current Exon = 2
>Nymphalis_urticae_Exon_2_LR989982.1_6508152_6508300_left_1_right_1
gaGGACACGTAACTTGAGTGAAAAGAAGAGGCGAGACCAATTCAATATGCTTGTCAGTGAACTTGGATCTATGGTTTCAACAAATAATAGAAAGATGGACAAGTCTACTGTCCTTAAATct

10738061 10738178
0 1
Getting Gene
Current Exon = 4
>Vanessa_cardui_Exon_4_NC_061154.1_10738061_10738178_left_0_right_1
agTGATATGGTGAacaaaagtatatttgaatTGGCATTTGATGATGATCGTTCTAACCTATACGCTTTACTTCAAAACCCTAGTATGGTGATACATCCTAATCAAACAATATCATCAG
10739865 10739960
2 1
Getting Gene
Current Exon = 5
>Vanessa_cardui_Exon_5_NC_061154.1_10739865_10739960_left_2_right_1
AGAACGAGATACGGTTCCAGTGTCATTTACAAAGGGGAACACTGGATTTTCGAGATGATATCACATACGAACTGGTTCAATTCAAAGGACATTTTC
10741351 10741411
2 2
Getting Gene
Current Exon = 6
>Vanessa_cardui_Exon_6_NC_061154.1_10741351_10741411_left_2_right_2
GTACGTATATGGAACAGAGTGATAGTTCCGACATCTTATACCAACAAGATCAGGAGTCAag
10744193 10744431
1 1
Getting Gene
Current Exon = 7
>Vanessa_cardui_Exon_7_NC_061154.1_10744193_10744431_left_1_right_1
GCTTCTGTTCGTCTGTACCGGGAAGCTGTGCACGCCTCAACTGATCCGAGATGTGTCTCTAGTTGACACAAGTCGCAGCGAGTTTACATCGCGTCACAGTTTGGAATGGAAATTCCTGTTCTTAGATCATCGCGCGCCGCCCATCATAGGTTACCTACCGTTTGAAGTGCTCGGTACTTCTGGATACGATTACTACCACTTCGATGATCTGGAGAAAGTGGTGACGTGTCATGAAGCAT
10